## Introduction 

Dans ce projet, nous allons travailler avec les données du CIA World Factbook, un recueil de statistiques sur tous les pays de la planète. Le Factbook contient des informations démographiques telles que les suivantes:

- population (La population mondiale)
- population_growth (le taux de croissance annuel de la population, en pourcentage)
- superficie (la superficie totale des terres et des eaux)

### Objectif

Pour ce projet, nous allons utiliser SQL pour effectuer une analyse de base des données 

In [ ]:
import sqlite3 as sq
import mysql.connector as cnt

### Se connecter à la base

In [ ]:
def connect_db(host, user, pswd, database = None):
    db = cnt.connect(
        host = host,
        user = user,
        passwd = pswd,
        database = database
    )
    return(db)

def exe(db, request, value = None, verbose = False, name = False, buffered = True):
    cursor = db.cursor(buffered = buffered)
    
    if value:
        cursor.execute(request, value)

    else :
        cursor.execute(request)

    db.commit()
    
    if verbose:
        result = cursor.fetchall()
        if name:
            result.insert(0, cursor.column_names)
        return(result)

#### Créer une connection à la base de données factbook.db

In [ ]:
# Connexion à Factbase.db
db_local = sq.connect("factbook.db")

# Récupération des données

query = "SELECT * FROM sqlite_master;"
create_table = exe(db_local, query, verbose = True, buffered = False)

query = "SELECT * FROM facts;"
data = exe(db_local, query, verbose = True, buffered = False)

db_local.close()

In [ ]:
# création de la BDD sur le server

db = connect_db("192.168.20.118", "grogu", "grogu")

exe(db, "CREATE DATABASE IF NOT EXISTS factbook")

In [ ]:
# Connexion à la BDD GRETA

db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

In [ ]:
# Création des deux tables

requete_1 = create_table[0][-1].replace('"', '`').replace('(name,seq)', '(name VARCHAR(255), seq INT)')
requete_2 = create_table[1][-1].replace('"', '`').replace('AUTOINCREMENT', 'AUTO_INCREMENT')

exe(db, f"{requete_1};")
exe(db, f"{requete_2};")
exe(db, "ALTER TABLE `facts` CHANGE `population` `population` BIGINT(12) NULL DEFAULT NULL;")

In [ ]:
# Ajout des données

for row in data:
    requete = "INSERT INTO facts VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
    values = [value for value in row]
    exe(db, requete, value = values)

**Vérification des 5 premières lignes du tableau des faits**

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = "Select * FROM facts WHERE id < 6;"

result = exe(db, query, verbose = True)
for line in  result:
    print(line)


### Descriptions des données
- name : Le nom du pays
- area : La superficie totale du pays (terre et eau)
- area_land : La superficie terrestre du pays en kilomètres carrés
- area_water : Superficie en eau du pays en kilomètres carrés
- population : Population du pays
- population_growth : Croissance de la population du pays en pourcentage
- birth rate : Le taux de natalité du pays, ou le nombre de naissances par an pour 1 000 habitants.
- death_rate : Taux de mortalité du pays, ou nombre de décès par an pour 1 000 personnes.


### Utilisation de sous-requêtes pour détecter les pays présentant ces valeurs démographiques aberrantes.


#### Résumé statistiques


In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

# Afficher le pays ayant la plus petite population 

query = "SELECT name FROM facts WHERE population = (SELECT MIN(population) FROM facts);"

result = exe(db, query, verbose = True)

print(f"Pays avec la plus petite population : {result[0][0]}\n")

# Afficher le pays ayant la plus grande population 

query = """SELECT name FROM facts WHERE population = (SELECT MAX(population) FROM facts WHERE name <> "World");"""

result = exe(db, query, verbose = True)

print(f"Pays avec la plus grande population : {result[0][0]}\n")

# Afficher le pays ayant le plus petit taux de natalité 

query = "SELECT name FROM facts WHERE birth_rate = (SELECT MIN(birth_rate) FROM facts);"

result = exe(db, query, verbose = True)

print(f"Pays avec le plus petit taux de natalité : {result[0][0]}\n")

# Afficher le pays ayant le plus grand taux de natalité 

query = "SELECT name FROM facts WHERE birth_rate = (SELECT MAX(birth_rate) FROM facts);"

result = exe(db, query, verbose = True)

print(f"Pays avec le plus grand taux de natalité : {result[0][0]}\n")

#### Afficher le nom et la population  du pays ayant la plus grande population

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = """SELECT name, population FROM facts WHERE population = (SELECT MAX(population) FROM facts WHERE name <> "World");"""

result = exe(db, query, verbose = True)

print(f"Pays avec la plus grande population : {result[0][0]}\nNombre d'habitant : {result[0][1]:,}")

####  Calculer la valeur moyenne de la population et de la superficie

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = """SELECT AVG(population) FROM facts WHERE name <> "World";"""

pop = exe(db, query, verbose = True)

query = "SELECT AVG(area) FROM facts;"

sup = exe(db, query, verbose = True)

print(f"Valeur moyenne de la population des pays du monde : {pop[0][0]:,}\nValeur moyenne de la surface des pays du monde : {sup[0][0]:,}")

#### Trouver les pays qui sont densément peuplés. 


I.E - Pays ayant des valeurs supérieures à la moyenne pour la population et inférieures à la moyenne pour la superficie.

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE (population > {round(pop[0][0], 0)} AND area < {round(sup[0][0], 0)} AND name <> "World");"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} est densément peuplé\n")

## Répondre à d'autres questions axées sur les données

#### 1. Quels pays sont les plus peuplés ? Quels pays ont le taux de croissance le plus élevé ? Nous restons dans la même logique de comparaison avec la moyenne

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE (population > {round(pop[0][0], 0)} AND name <> "World");"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} est plus peuplé que la moyenne\n")

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE (population_growth > (SELECT AVG(population_growth) FROM facts WHERE name <> "World") AND name <> "World");"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} a une croissance de population plus importante que la moyenne\n")

#### 2.1 Quels pays ont les rapports eau/terre les plus élevés ?

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE (area_water / area_land) > (SELECT AVG(area_water / area_land) FROM facts);"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} a un rapport eau/terre plus importante que la moyenne\n")

#### 2.2 Quels pays ont plus d'eau que de terre ?

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE (area_water > area_land);"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} a plus d'eau que de terre\n")

#### 3. Quels sont les 5 pays dont la population augmentera le plus l'année prochaine ?
I.E Calculer la croissance démographique effective (population_growth * population) pour une année sur la base de la population actuelle du pays.

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE name <> "World" ORDER BY (population_growth * population) DESC;"""

result = exe(db, query, verbose = True)

for country in result[:5] :
    print(f"{country[0]} est un pays dont la population augmentera le plus l'année prochaine \n")

#### 4. Quels pays ont un taux de mortalité supérieur au taux de natalité ? (Taux à afficher)

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name, death_rate, birth_rate FROM facts WHERE ((death_rate > birth_rate) AND name <> "World");"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} est un pays qui a un taux de mortalité supérieur au taux de natalité avec un rapport de {country[1]}/{country[2]} \n")

#### 5. Quels sont les pays dont le rapport population / superficie est le plus élevé ?

In [ ]:
db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

query = f"""SELECT name FROM facts WHERE ((population / area) > (SELECT AVG(population / area) FROM facts) AND name <> "World") ORDER BY (population / area) DESC;"""

result = exe(db, query, verbose = True)

for country in result :
    print(f"{country[0]} est un pays qui a rapport population / superficie élevé\n")

# Remettre les données dans un dataframe à l'aide de Pandas et répondre aux questions précédement posées

In [ ]:
import pandas as pd

db = connect_db("192.168.20.118", "grogu", "grogu", "factbook")

liste = [list(line) for line in exe(db, "SELECT * FROM facts", name = True, verbose = True)]

df = pd.DataFrame(data = liste[1:], columns = liste[0])
df

#### 1. Quel est le pays le plus peuplé ? Quel pays a le taux de croissance le plus élevé ? Nous restons dans la même logique de comparaison avec la moyenne

In [ ]:

result = df[df['population'] > df['population'].mean()]['name']
display(result)

In [ ]:
result = df[df['population_growth'] > df['population_growth'].mean()]['name']
display(result)

#### 2.1 Quels pays ont les rapports eau/terre les plus élevés ?

In [ ]:
result = df[(df['area_water'] / df['area_land']) > (df['area_water'] / df['area_land']).mean()]['name']
display(result)

#### 2.2 Quels pays ont plus d'eau que de terre ?

In [ ]:
result = df[df['area_water'] > df['area_land']]['name']
display(result)

#### 3. Quels sont les 5 pays dont la population augmentera le plus l'année prochaine ?

In [ ]:
result = df.iloc[(df['population']*df['population_growth']).sort_values(ascending = False).head(6).index]['name'].iloc[1:6]
print(result)

#### 4. Quels pays ont un taux de mortalité supérieur au taux de natalité ? (Taux à afficher)

In [ ]:
result = df[df['death_rate'] > df['birth_rate']][['name', 'death_rate', 'birth_rate']]
display(result)

#### 5. Quels sont les pays dont le rapport population / superficie est le plus élevé ?

In [ ]:
result = df.iloc[(df['population']/df['area']).sort_values(ascending=False).index]['name']
print(result)